In [1]:
import parflow as pf
import numpy as np
from osgeo import gdal

#### Read in pfb

In [3]:
# inputs for tiff
path_to_dir = '/glade/scratch/tijerina/CONUS2/spinup_WY2003/run_inputs'
run_name = 'spinup.wy2003'
timestep = '00216'
variable = 'press'


# Out directory to save tiff
out_dir = '/glade/work/tijerina/Figures/tiffs'

In [4]:
file = f'{path_to_dir}/{run_name}.out.{variable}.{timestep}.pfb'
pfb = pf.read_pfb(file)
#pfb = pf.read_pfb('/glade/scratch/tijerina/from_hoang/CONUS2_spinup_all_seepage.out.press.08754.pfb')
print(file)
pfb.shape


/glade/scratch/tijerina/CONUS2/spinup_WY2003/run_inputs/spinup.wy2003.out.press.00216.pfb


(10, 3256, 4442)

In [5]:
# select layer (only for outputs with multiple layers)
pfb10 = pfb[9,...]
#### Flip pfb array to make tif
# need to do this because read_pfb starts indexing at top left and we need it to start at bottom left

pfb_flip = np.flipud(pfb10) # USE FOR SINGLE LAYER: np.flipud(pfb)

In [6]:
# Set inactive domain to Nan
pfb_flip[pfb_flip < -1000000000] = 'Nan'

#### Read in CONUS2 file for georeferencing information

In [9]:
# this file is needed for the geotransform and projection below
# this land cover file is in a directory with other tiffs, but it may be best to cp it into your working directory
file = '/glade/work/tijerina/Figures_NotebookOutputs/tiffs/1km_CONUS2_landcover_IGBP.tif'

# open tiff mask
lc_tif = gdal.Open(file)
# check to make sure file was read correctly as a osgeo.gdal dataset
type(lc_tif)

osgeo.gdal.Dataset

In [8]:
### Change the destination file name, what you would like your new tif to be called ###
dst_filename = f'{out_dir}/{run_name}_{variable}.{timestep}.tiff'
#dst_filename = 'Hoang_init_pressure.tiff'
### number of grid cells in x ###
x_pixels = 4442  
### number of grid cells in y ###
y_pixels = 3256  

driver = gdal.GetDriverByName('GTiff')
dataset = driver.Create(dst_filename,x_pixels, y_pixels, 1,gdal.GDT_Float32)

### create tif with FLIPPED array you want to use ###
dataset.GetRasterBand(1).WriteArray(pfb_flip) 

# Do not change code below here, unless you have used a different variable name above for the land cover tif
# Add GeoTranform and Projection information to newly created tif file
geotrans=lc_tif.GetGeoTransform()  #get GeoTranform from existing tif file (here 'lc_tif')
proj=lc_tif.GetProjection() #you can import or get from a existing tif   
dataset.SetGeoTransform(geotrans)
dataset.SetProjection(proj)
dataset.FlushCache()
dataset=None